In [3]:
import xlwings
import sqlite3
import math
import pandas as pd
from xlwings.ext.sql import sql
import time
from datetime import datetime

## Excel Functions

In [1]:
def openXLfile(filepath):
    excel_app = xlwings.App(visible=False)
    excel_book = excel_app.books.open(filepath)
    return excel_app, excel_book

# ------------------------------------------------------------------------------------------------------------------------------    

def getParam(fsd_sheet, dist_wb, source, rrup, site_to_source_az):
    """ Gets parameters required for GMPE from fault section data """
    sheet = fsd_sheet
    t1 = sheet.range("A1:H351").value
    
    source = source + '%'
    query = sql(f"""SELECT * From a 
                WHERE a.Name LIKE '{source}';""", t1)

    
    if len(query) == 1:
        # 1 default entry (with attribute names) will always be there in every query, 
        print("Fault Section Data not available")

    elif len(query) > 2:
        print("More than one fault section data available")


    fault_data = query[1]
    dip = fault_data[4]
    ztor = fault_data[5]
    z_bottom = fault_data[6]

    vert_depth = z_bottom - ztor

    dip_rad = dip * math.pi/180
    w = vert_depth / math.sin(dip_rad)
    
    source_to_site_az = site_to_source_az - 180

    rjb, rx = getDist(dist_wb, ztor, w, dip, source_to_site_az, rrup)

    return rjb, rx, dip

# ------------------------------------------------------------------------------------------------------------------------------    

def getDist(dist_wb, ztor, w, dip, source_to_site_az, rrup):
    """Estimates distance parameters (R_jb and R_x)"""
    book = dist_wb
    sheet = book.sheets["Distance Equations"]

    ztor_cell = sheet.range("B31")
    w_cell = sheet.range("B32")
    dip_cell = sheet.range("B33")
    rjb_cell = sheet.range("B36")
    source_to_site_az_cell = sheet.range("B37")
    rx_cell = sheet.range("B40")
    rrup_cell = sheet.range("B41")
    target_rrup_cell = sheet.range("J41")

    # Assignment
    ztor_cell.value = ztor
    w_cell.value = w
    dip_cell.value = dip
    source_to_site_az_cell.value = source_to_site_az
    target_rrup_cell.value = rrup

    gs = book.macro('gs')
    gs.run()

    rjb = rjb_cell.value
    rx = rx_cell.value
    
    return rjb, rx

# ------------------------------------------------------------------------------------------------------------------------------    

def writeInExcel(db_path):
    """Write the entries of interest in an excel sheet (Writes in Fragility.xlsm) """
    db = sqlite3.connect(db_path, timeout = 10)

    query = pd.read_sql("Select * from EntryofInterest Order by Loc_id, source_name", db)

    db.close()

    app, book = openXLfile("pyExcel_Sheets/Fragility.xlsm")

    sheet = book.sheets["HazardData"]
    sheet.clear()


    
    sheet["A1"].options(pd.DataFrame, header=1, index=True, expand='table').value = query
    
    sheet["A1"].expand("right").api.Font.Bold = True
    sheet["A1"].expand("down").api.Font.Bold = True
    sheet["A1"].expand("right").api.Borders.Weight = 2
    sheet["A1"].expand("down").api.Borders.Weight = 2
    sheet["A1"].expand("right").api.WrapText = True
    
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    sheet["A1"].value = "Updated on: " + dt_string

    book.save()

    app.kill()
    return


### Functions dealing with GMPE.xlsm 

In [1]:

def view_value():
    print(sheet.range("C14").value)
    print(sheet.range("D14").value)
    print(sheet.range("E14").value)
    print(sheet.range("F14").value)
    print(sheet.range("G14").value)


def check_weights(sheet):
    cell = sheet.range("C14")
    
    # Set first 4 GMM weight as 0.22
    for i in range(4):
        cell.offset(0, i).value = 0.22
    
    # Set Idriss GMM weight as 0.12
    cell.offset(0, 4).value = 0.12
#     view_value()
    return


def get_pga(sheet):
    return sheet.range("F45").value

def get_sa0p2(sheet):
    return sheet.range("F30").value

def get_sa0p3(sheet):
    return sheet.range("F32").value

def get_sa1p0(sheet):
    return sheet.range("F36").value
    

    
def xl_update(sheet, m, r_rup, r_jb, r_x, dip, vs30):
    check_weights(sheet)
    
    sheet.range("B24").value = m
    sheet.range("B27").value = r_rup
    sheet.range("B30").value = r_jb
    sheet.range("B33").value = r_x
    sheet.range("B39").value = vs30
    sheet.range("B54").value = dip
    

    sheet.range("B36").value = 999 
    sheet.range("B42").value = 0
    sheet.range("B45").value = 0
    sheet.range("B48").value = 0
    sheet.range("B51").value = 0
    return
    

In [3]:
def executeHazardAlgorithm(db_path, vs30):
    # Inputs 
    start = time.time()
    gmpe_filepath = "pyExcel_Sheets/GMPE.xlsm"
    fsd_filepath = "pyExcel_Sheets/Fault_Section_Data.xlsx"
    dist_filepath = "pyExcel_Sheets/dist.xls"


    # Process
    db = sqlite3.connect(db_path, timeout = 10)
    cur = db.cursor()

    cur.execute("SELECT * FROM EntryOfInterest")
    query = cur.fetchall()
    cur.close()


    # Open all required Excel sheets
    gmpe_xl_app, gmpe_wb = openXLfile(gmpe_filepath)
    gmpe_ws = gmpe_wb.sheets["Main"]

    fsd_xl_app, fsd_wb = openXLfile(fsd_filepath)
    fsd_ws = fsd_wb.sheets["GeometryData"]

    dist_xl_app, dist_wb = openXLfile(dist_filepath)


    cur = db.cursor()
    for row in query:
        imt = row[2]
        source = row[5]
        rrup = row[6]
        m = row[7]
        site_to_source_az = row[11]
        rjb, rx, dip = getParam(fsd_ws, dist_wb, source, rrup, site_to_source_az)

        xl_update(gmpe_ws, m, rrup, rjb, rx, dip, vs30)

        haz_values = {'PGA': get_pga(gmpe_ws),
               'SA0P2': get_sa0p2(gmpe_ws),
               'SA1P0': get_sa1p0(gmpe_ws)}
        haz = haz_values[imt]   

        SA0P3 = get_sa0p3(gmpe_ws)
        SA1P0 = haz_values['SA1P0']


        query = f"""UPDATE EntryOfInterest
                    SET hazard = {haz}, SA0P3 = {SA0P3}, SA1P0 = {SA1P0}
                    WHERE (loc_id, return_period, imt, source_set, source_name) IS {row[0:5]}"""

        cur.execute(query)

    cur.close()
    db.commit()

    gmpe_xl_app.kill()
    fsd_xl_app.kill()
    dist_xl_app.kill()

    db.close()
    
    # Write the entries of interest in an excel sheet (Writes in Fragility.xlsm)
    writeInExcel(db_path)


    # ------------------------------------------------------------------------------------------------------------------------------    
    end = time.time()
    return

